In [6]:
import os, sys
import traceback
import pandas as pd
import re
import glob
import time
from string import ascii_uppercase
from natsort import natsorted, index_natsorted
import math
import numpy as np
import shutil

print('Working directory: \t',os.getcwd(), '\n')


def write_to_subfolder(result_df, new_file, subdirectory):
    
    curr_dir = os.getcwd()
    new_dir = os.path.join(curr_dir,subdirectory)
    counter = 0
    
    if curr_dir.find(subdirectory) == -1:
        
        try:
            os.mkdir(subdirectory)
        except Exception:
            pass
    
        os.chdir(new_dir)
        print('File written  :-  ', new_file)
        result_df.to_csv(new_file, index = False)
        counter = 1
        os.chdir('..')
        
    else:
        os.chdir(new_dir)
        print('File written  :-  ', new_file)
        result_df.to_csv(new_file, index = True)
        os.chdir('..')
        

def Read_primer_file():

    df_all = pd.DataFrame()
    df_temp = pd.DataFrame()

    filenames = glob.glob('**/*bvmo_primers_384wellIDT*.csv')
    
    for file in filenames:
        
        df_temp = pd.read_csv(file)
        print('Filename: \t',file, '\n')
        
        df_PCR_Rev, df_PCR_Fwd = PCR_Set_up(df_temp)
        
        df_PCR_Fwd = Add_96well_dest(df_PCR_Fwd, 'PCR_I_Fwd_')
        df_PCR_Rev = Add_96well_dest(df_PCR_Rev, 'PCR_II_Rev_')
        
        #Mix_Fwd_Rev_PCRs(df_PCR_Fwd, df_PCR_Rev)
        Mix_Fwd_Rev_PCRs_simple(df_PCR_Fwd, df_PCR_Rev)
        
    
        
        
def PCR_Set_up(df_Primers):
    
    column_names = ["Source Well", "Transfer Volume", "Destination Well", "Primer Name"]
    df_PCR_Fwd = pd.DataFrame(columns = column_names) 
    df_PCR_Rev = pd.DataFrame(columns = column_names)    
    List1 = []
    
    template_counter = 1
    Vol_primer = 2500
    Vol_template = 1000
    
    for index, row in df_Primers.iterrows():
        
        if index == len(df_Primers)-1:
            break
            
        Wellnumber = math.ceil(template_counter / 80)
        template_Well = 'P'+str(Wellnumber)
        template_counter = template_counter + 1

        # The rev primer should come after Fwd primer
        Fwd_primer = df_Primers['Name'].loc[index]
        Rev_primer = df_Primers['Name'].loc[index+1]
        
        Fragment_fwd = re.findall('\_(.*?)\_', Fwd_primer)
        Fragment_rev = re.findall('\_(.*?)\_', Rev_primer)
        
        Fwd_primer_name = Fragment_fwd[0] + '_F'
        Rev_primer_name = Fragment_rev[0] + '_R'
        
        isFwd = ((re.search(Fwd_primer_name, Fwd_primer))) 
        isRev = ((re.search(Rev_primer_name, Rev_primer))) 
        
        if isFwd and isRev and (Fragment_fwd[0] == Fragment_rev[0]):
            
            S_well_1 = df_Primers['Well Position'].loc[index]
            S_well_2 = df_Primers['Well Position'].loc[index+1]
            d_well = '' # will add later
            
            List1 = [S_well_1, Vol_primer, d_well, Fwd_primer]
            a_series = pd.Series(List1, index = df_PCR_Fwd.columns)
            df_PCR_Fwd = df_PCR_Fwd.append(a_series, ignore_index=True)
            List1.clear()
            
            List1 = [S_well_2, Vol_primer, d_well, Rev_primer]
            a_series = pd.Series(List1, index = df_PCR_Rev.columns)
            df_PCR_Rev = df_PCR_Rev.append(a_series, ignore_index=True)
            List1.clear()

            List1 = [template_Well, Vol_template, d_well, '']
            a_series = pd.Series(List1, index = df_PCR_Rev.columns)
            df_PCR_Rev = df_PCR_Rev.append(a_series, ignore_index=True)
            df_PCR_Fwd = df_PCR_Fwd.append(a_series, ignore_index=True)
            List1.clear()
            
        else:
            continue
               
    df_PCR_Fwd = df_PCR_Fwd.reset_index(drop = True)
    df_PCR_Rev = df_PCR_Rev.reset_index(drop = True)

    return df_PCR_Rev, df_PCR_Fwd



def Wellplate96_PCR():
    
    # we will keep wells A1, A2 and H11, H12 empty
    # For running on FA, to add ladder or some other standrd
    # In each 96well plate, we will only set 92 PCRs
    #Skip_wells = ['A1', 'A2', 'H11', 'H12']

    Wellplate96 = []

    for letter in ascii_uppercase:
        for j in range(1,13):            
            dest = letter+str(j)
            #if dest in Skip_wells:
             #   continue
            Wellplate96.append(dest)
        if letter == 'H':
            break
            
    return Wellplate96

    
def Add_96well_dest(df_PCR, PCR_type):
    
    df_PCR = df_PCR.reset_index(drop = True)

    Wellplate96 = Wellplate96_PCR()
        
    PCR_per_plate = 96
    Dest_well_list = []
    
    number_of_PCR = int(len(df_PCR) / 2)
        
    for PCR in range(number_of_PCR):   
        
        if (PCR % PCR_per_plate) == 0:
            index = 0
        
        for i in range(2):
            Dest_well_list.append(Wellplate96[index])
            
        index = index + 1

    df_PCR['Destination Well'] = Dest_well_list
    filename = PCR_type+'.csv'
    write_to_subfolder(df_PCR, filename, 'PCR_setup')
    
    return df_PCR
        
    
    
def Mix_Fwd_Rev_PCRs_simple(df_PCR_Fwd, df_PCR_Rev):
    
    column_names = ["Source plate", "S-Well position", "Destination plate", "D-Well position", "Volume", "Primer"]
    df_Fluent_temp = pd.DataFrame(columns = column_names)
    df_Fluent_mixing = pd.DataFrame(columns = column_names)
    
    Wellplate96 = Wellplate96_PCR()

    if len(df_PCR_Fwd) == len (df_PCR_Rev):
        Number_of_PCRs = int(len(df_PCR_Fwd)/2)
    
    if Number_of_PCRs > 96:
        print('\t ERROR!!!   Number of PCRs should be <= 96')
        
    Dest_Well = Wellplate96[:Number_of_PCRs]
    
    df_Fluent_temp = Create_PCR_mixing_Picklist(df_PCR_Fwd, Dest_Well, df_Fluent_temp, 'PCR_Fwd_plate_001' )    
    df_Fluent_mixing = Create_PCR_mixing_Picklist(df_PCR_Rev, Dest_Well, df_Fluent_mixing,'PCR_Rev_plate_002' )

    df_Fluent_mixing = df_Fluent_temp.append(df_Fluent_mixing)
    df_Fluent_mixing = df_Fluent_mixing.reset_index(drop=True)
    
    df_Fluent_mixing_for_gwl = df_Fluent_mixing.reset_index(drop = True)

    
    df_Fluent_mixing = df_Fluent_mixing.sort_values(by=['D-Well position'], 
            key=lambda x: np.argsort(index_natsorted(df_Fluent_mixing["D-Well position"])))

    df_Fluent_mixing = df_Fluent_mixing.reset_index(drop = True)
    
    flag = Validate_PCR_mixing_picklist(df_Fluent_mixing)

    write_to_subfolder(df_Fluent_mixing, 'Mix_PCRs_Fluent.csv', 'PCR_setup')
    
    if flag == True:
        print('\n The picklist above ^^ has been validated \n ')
        create_gwl_worklist(df_Fluent_mixing_for_gwl)

    else:
        Print('\n ERRORs in above ^^ Fluent picklist  \n')
        

    
    
def Create_PCR_mixing_Picklist(df_PCR, Dest_Well, df_Fluent_mixing, Source_plate):
    
    df_PCR = df_PCR.rename({'Primer Name': 'Primer_name'}, axis = 1)
        
    df_PCR = df_PCR[df_PCR.Primer_name != '']
    df_PCR = df_PCR.reset_index(drop=True)

    df_Fluent_mixing["S-Well position"] = Dest_Well
    df_Fluent_mixing["Destination plate"] = 'PCR_mix_plate_003'
    df_Fluent_mixing["D-Well position"] = Dest_Well
    df_Fluent_mixing["Volume"] = 25
    df_Fluent_mixing["Primer"] = df_PCR['Primer_name']
    df_Fluent_mixing["Source plate"] = Source_plate
    
    return df_Fluent_mixing

    
def Validate_PCR_mixing_picklist(df_Fluent):
    
    flag = -1
    
    for index, row in df_Fluent.iterrows():
        
        if index == len(df_Fluent)-1:
            break
            
        # The rev primer should come after Fwd primer
        Fwd_primer = df_Fluent['Primer'].loc[index]
        Rev_primer = df_Fluent['Primer'].loc[index+1]
        
        F_Swell = df_Fluent['S-Well position'].loc[index]
        R_Swell = df_Fluent['S-Well position'].loc[index+1]
        
        F_Dwell = df_Fluent['D-Well position'].loc[index]
        R_Dwell = df_Fluent['D-Well position'].loc[index+1]
        
        Frag_fwd = re.findall('\_(.*?)\_', Fwd_primer)
        Frag_rev = re.findall('\_(.*?)\_', Rev_primer)
        
        Fwd_primer_name = Frag_fwd[0] + '_F'
        Rev_primer_name = Frag_fwd[0] + '_R'
        
        isFwd = ((re.search(Fwd_primer_name, Fwd_primer))) 
        isRev = ((re.search(Rev_primer_name, Rev_primer))) 
        
        if isFwd and isRev and (F_Swell == R_Swell) and (F_Dwell == R_Dwell):
            
            if flag == 1:
                print(Fwd_primer_name, Rev_primer_name, 'Something wrong here')
                return False
            
            flag = 1
            
        else:
            
            if flag == -1:
                print(Fwd_primer_name, Rev_primer_name, 'Something wrong here')
                return False
            
            flag = -1
    
    return True

    
def create_gwl_worklist(Fluent_picklist):
    
    Step_list =[]
    
    for i in range(len(Fluent_picklist)):

        Volume = 25
        index = i
        counter = 0
        
        S_plate = Fluent_picklist.iloc[i, 0]
        S_well = Fluent_picklist.iloc[i, 1]
        Step_1 = str('A;'+ S_plate+';;;'+str(S_well)+';;'+str(Volume)+';;;;\n')
        Step_list.append(Step_1)

        D_plate = Fluent_picklist.iloc[i, 2]
        D_well = Fluent_picklist.iloc[i, 3]
        Step_2 = str('D;'+ D_plate+';;;'+str(D_well)+';;'+str(Volume)+';;;;\n')
        Step_list.append(Step_2)

        Step_list.append('W;\n')

        index = index + 1
            
    Gwl_Filename = 'Gwl_worklist_mixing_PCR.gwl'
    
    F = open(Gwl_Filename, "w+")
    F.writelines(Step_list)
    F.close()
        
    
    
def Move_GWL_files_to_subfolder():
    
    source = os.getcwd()
    Dest_dir = os.path.join(source, 'gwl_worklists')
    print('\n Moving all gwl flies to:  \t', Dest_dir, '\n')

    if not os.path.exists(Dest_dir):
        #shutil.rmtree(Dest_dir)
        os.makedirs(Dest_dir)

    gwl_files = glob.glob('*.gwl')

    for file in gwl_files:
        file_name_source = os.path.join(source, file)
        file_name_dest = os.path.join(Dest_dir, file)

        shutil.move(file_name_source, file_name_dest)

    
    
        
if __name__ == '__main__':
    
    Read_primer_file()
    Move_GWL_files_to_subfolder()




Working directory: 	 /Users/nilmani/Desktop/Protein_Engineering 

Filename: 	 IDT_Ordering/bvmo_primers_384wellIDT.csv 

File written  :-   PCR_I_Fwd_.csv
File written  :-   PCR_II_Rev_.csv
File written  :-   Mix_PCRs_Fluent.csv

 The picklist above ^^ has been validated 
 

 Moving all gwl flies to:  	 /Users/nilmani/Desktop/Protein_Engineering/gwl_worklists 

